# Algorithms for Data Science

## Counting Distinct Items

### 1. Preliminaries 

The objective of this lab is to implement the Flajolet-Martin approach to count distinct items. First, we generate an universe of $N$ strings of length $12$, and take $d$ items which will constitute our universe of distinct items.

In [1]:
import random
from string import ascii_lowercase

#parameters
N = 256 #universe of N 
d = 25 #distinct items
stream_size = 10000

#generate some random strings of size 10
U = []
for _ in range(N):
  U.append(''.join(random.choice(ascii_lowercase) for i in range(12)))

D = random.sample(U,k=d)

print(D)

['piqbyjtznjek', 'uaxeqszogogn', 'thrhbpehbexv', 'djueosejhmuj', 'engrporeueko', 'ksoatdfcpklu', 'yngoatdtsyhe', 'exnzyryfgdpv', 'auxuaxeooarw', 'xdxhzuklobhm', 'yaqrhxagnyfx', 'elehxaytunjl', 'vyfpngmekfmt', 'vdsbltoaqlpl', 'nozgizybfltb', 'oencvfsiigsj', 'nmdlzczklyyw', 'aoisxxfzlyym', 'syokygxbiaps', 'svtzxdevsrim', 'illjopurjkcm', 'lyfpcgmjfrvu', 'nbdyxwagupwz', 'bsdzqpsapdba', 'apsnztibxmqz']


### 2. Flajolet-Martin: Creating a Hash Function, Estimating Distinct Items Using Trailing 0s

In the following we create a hash function $h(x)$, which also takes as a parameter a hashable and $N$, and returns a value in $0,\dots,N-1$. We simulate a stream taking random values from $D$, count the trailing $0$s in its hash value, keep the maximum value $R$, and then output $2^R$ as the estimator.

In [2]:
import math

def h(x,n):
  return hash(x)%n

#method for counting trailing 0s
def trailing_0(x):
  x1 = x
  t0 = 0
  while x1%2==0 and x1!=0:
    t0 += 1
    x1 = int(x1/2)
  return t0

#simulating the stream
R = 0
for _ in range(stream_size):
  #take a random string from the distinct pool
  s = random.choice(D)
  #check its hash value
  hv = h(s,2*N) #to allow more space for hash values
  r = trailing_0(hv)
  if r>R: R=r

est = int(math.pow(2,R))

print('Estimation of distinct items: %d'%est)

Estimation of distinct items: 16


### 3. **TASK** Flajolet-Martin: Using Multiple Hash Functions

Implement the refined version of the above estimator, using multiple ($k$) hash functions (use the method of generating several pairs of numbers presented last time in the lab) and compute:
1. The average of the $k$ estimators
2. The median of the $k$ estimators
3. Divide the estimators into groups (vary the group size); take the median in each group and then the average over the groups.

Compare the three methods' final outputs. What do you notice?

_Note_: you can use the Python 3.4 _statistics_ package (not available in previous versions) to compute medians, averages, and other statistics.

In [4]:
# YOUR CODE HERE
import math
from statistics import mean, median


def h1(x,n):
  return (hash(x) + 1)%n

def h2(x,n):
  return (2 *hash(x) + 1)%n

def h3(x,n):
  return (hash(x) - 3)%n

k = 3
p = 1223543677
n = 2*N

def h(x):
  return ((a*hash(x)+b)%p)%n

h_list = list()


for i in range(k):
    a = random.randrange(p)
    b = random.randrange(p)
    h_list.append(h)

#simulating the stream
R = [0] * k
for _ in range(stream_size):
    #take a random string from the distinct pool
    s = random.choice(D)
    #check its hash value
    
    for i in range(k):
        hv = h_list[i](s)
        r = trailing_0(hv)
        R[i] = max(R[i], r)

est_list = [math.pow(2,r) for r in R]
mean_est = int(mean(est_list))
median_est = int(median(est_list))

half = len(est_list) // 2
hybrid_est = median((mean(est_list[:half]), mean(est_list[half:])))

print('Estimation of distinct items using mean: %d'%mean_est)
print('Estimation of distinct items using median: %d'%median_est)
print('Estimation of distinct items using hybrid: %d'%hybrid_est)

Estimation of distinct items using mean: 256
Estimation of distinct items using median: 256
Estimation of distinct items using hybrid: 256


_You can use this cell to write your discussion of the results_